<div align="center">
 <b>HOMEWORK</b>
</div>
In this homework, we will use the Bank Marketing dataset.
wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
We need to take bank/bank-full.csv file from the downloaded zip-file.

In [44]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression



In [62]:
df_full = pd.read_csv('bank-full.csv', delimiter=';')

# In this dataset our desired target for classification task will be y variable - has the client subscribed a term deposit or not.
# For the rest of the homework, you'll need to use only these columns:
# age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y

# 1. Select only the features from above.
features = [
    'age',
    'job',
    'marital',
    'education',
    'balance',
    'housing',
    'contact',
    'day',
    'month',
    'duration',
    'campaign',
    'pdays',
    'previous',
    'poutcome',
]

df_features = df_full[features]
df_target = df_full['y']

df = pd.concat([df_features, df_target], axis=1)
print(len(df))

# 2. Check if the missing values are present in the dataset.
missing_values_features = df_features.isnull().sum()
print(f'Missing values in features:\n{missing_values_features}')
print('\n')
missing_value_target = df_target.isnull().sum()
print(f'Missing values in target:\n{missing_value_target}')



45211
Missing values in features:
age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
dtype: int64


Missing values in target:
0


In [26]:
## Question 1
# What is the most frequent observation (mode) for the column education?

mode_education = df_features['education'].mode()[0]
print(f' The mode for education is: {mode_education}')

 The mode for education is: secondary


In [32]:
## Question 2
# Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.
# What are the two features that have the biggest correlation?

# Select only numeric columns
numeric_df_features = df_features.select_dtypes(include=[float, int])
# Compute the correlation matrix
correlation_matrix = numeric_df_features.corr()
# Check the two features with the biggest correlation
# Unstack the correlation matrix to get a Series
correlation_series = correlation_matrix.unstack()
# Remove self-correlation (correlation of a feature with itself)
correlation_series = correlation_series[correlation_series.index.get_level_values(0) != correlation_series.index.get_level_values(1)]
# Sort the Series by the absolute value of the correlation coefficients
sorted_correlation_series = correlation_series.abs().sort_values(ascending=False)

# Get the pair of features with the highest correlation
highest_correlation_pair = sorted_correlation_series.idxmax()
highest_correlation_value = sorted_correlation_series.max()
print(f"The two features with the highest correlation are: {highest_correlation_pair} with a correlation of {highest_correlation_value}")



The two features with the highest correlation are: ('previous', 'pdays') with a correlation of 0.4548196354805043


In [34]:
# Target encoding
# Now we want to encode the y variable.
# Let's replace the values yes/no with 1/0.

df_target = df_target.replace({'yes': 1, 'no': 0})
df_target.head()

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64

In [57]:
# Split your data in train/val/test sets with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
# Make sure that the target value y is not in your dataframe.

X_train_val, X_test = train_test_split(df, test_size=0.2, random_state=42)
X_train, X_val = train_test_split(X_train_val, test_size=0.25, random_state=42)

print(len(df_features))
print(len(X_train) + len(X_val) + len(X_test))
print(len(X_train), len(X_val), len(X_test))

45211
45211
27126 9042 9043


In [82]:
## Question 3
# Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
# Round the scores to 2 decimals using round(score, 2).
# Which of the following features has the highest mutual information score with the target variable y?

# Select only categorical columns
X_train_categorical = X_train.select_dtypes(include=[object])
Y_train = X_train['y'].replace({'yes': 1, 'no': 0})
X_train_categorical = X_train_categorical.drop(columns='y')


# Select only categorical columns names in a list
#categorical_columns = X_train.select_dtypes(include='object').columns.tolist()

# Calculate the mutual information score between y and each categorical feature
mutual_info_scores = []
for column in X_train_categorical.columns:
    score = mutual_info_score(X_train_categorical[column], Y_train)
    mutual_info_scores.append((column, score))

# Convert the list of tuples to a DataFrame for sorting and rounding
mi_scores_df = pd.DataFrame(mutual_info_scores, columns=['Feature', 'MI Score'])

# Sort by highest mutual information score and round to 2 decimals
mi_scores_df = mi_scores_df.sort_values(by='MI Score', ascending=False).round(2)
highest_mi_score = mi_scores_df.iloc[0]

# Display the feature with the highest mutual information score
print(f' The Variable with biggest mutual information score is:\n\n{highest_mi_score[0]} with a score of {highest_mi_score[1]}')

# one hot encoding with feature-engine (my preferred method)
# from feature_engine import encoding
# onehot = encoding.OneHotEncoder(variables=categorical_columns)
# onehot.fit(X_train_categorical)
# X_train_categorical_feature = onehot.transform(X_train_categorical)
# X_train_categorical_feature

# one hot encoding with DictVectorizer method
# dv = DictVectorizer(sparse=False)
# X_train_categorical_dict = X_train_categorical.to_dict(orient='records')
# X_train_categorical_encoded = dv.fit_transform(X_train_categorical_dict)
# X_train_categorical_encoded = pd.DataFrame(X_train_categorical_encoded, columns=dv.get_feature_names_out())
# X_train_categorical_encoded

 The Variable with biggest mutual information score is:

poutcome with a score of 0.03


C:\Users\gabriel.rangel\AppData\Local\Temp\ipykernel_17084\1745777648.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y_train = X_train['y'].replace({'yes': 1, 'no': 0})
C:\Users\gabriel.rangel\AppData\Local\Temp\ipykernel_17084\1745777648.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f' The Variable with biggest mutual information score is:\n\n{highest_mi_score[0]} with a score of {highest_mi_score[1]}')


In [91]:
## Question 4
# Now let's train a logistic regression.
# Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
# Fit the model on the training dataset.
# To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
# model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

x_train = X_train.drop(columns='y')
x_val = X_val.drop(columns='y')

# one hot encoding with DictVectorizer method
X_train_dict = x_train.to_dict(orient='records')
X_val_dict = x_val.to_dict(orient='records')
#X_test_dict = X_test.to_dict(orient='records')

# Initialize the DictVectorizer
dv = DictVectorizer(sparse=False)

# Apply DictVectorizer to the entire dataset (both numerical and categorical features)
X_train_encoded = dv.fit_transform(X_train_dict)
X_val_encoded = dv.transform(X_val_dict)
#X_test_encoded = dv.transform(X_test_dict)

# Train the logistic regression model with the specified parameters
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, Y_train) # Y_train is the target variable that we set on previous exercise

# # We need the Y_val to calculate the accuracy
Y_val = X_val['y'].replace({'yes': 1, 'no': 0})

# Calculate the accuracy on the validation dataset and round it to 2 decimal digits
# (Option 1)
original_accuracy = model.score(X_val_encoded,Y_val)
accuracy_rounded = round(original_accuracy, 2)


# Option 2 - using the predict method and accuracy_score
# from sklearn.metrics import accuracy_score
# # Make predictions on the validation set
# y_pred = model.predict(X_val_encoded)

# # Calculate accuracy and round to 2 decimal places
# original_accuracy = accuracy_score(Y_val, y_pred)
# accuracy_rounded = round(original_accuracy, 2)

print(f"Validation Accuracy: {accuracy_rounded}")


Validation Accuracy: 0.9


C:\Users\gabriel.rangel\AppData\Local\Temp\ipykernel_17084\156047848.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y_val = X_val['y'].replace({'yes': 1, 'no': 0})


In [100]:
## Question 5
# Let's find the least useful feature using the feature elimination technique.
# Train a model with all these features (using the same parameters as in Q4).
# Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
# For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
# Which of following feature has the smallest difference? (age,balance,marital or previous)

# Store the differences in accuracy
accuracy_differences = []

# Iterate over the features and remove one at a time
for feature in dv.get_feature_names_out():
    # Get the index of the feature to exclude
    feature_index = [i for i, f in enumerate(dv.get_feature_names_out()) if f == feature]

    # Create new datasets without the specified feature
    X_train_reduced = np.delete(X_train_encoded, feature_index, axis=1)
    X_val_reduced = np.delete(X_val_encoded, feature_index, axis=1)

    # Train the model without the feature
    model.fit(X_train_reduced, Y_train)

    accuracy = model.score(X_val_reduced,Y_val)


    # Calculate the absolute difference in accuracy
    accuracy_diff = abs(original_accuracy - accuracy)
    accuracy_differences.append((feature, round(accuracy_diff, 4)))

# Find the feature with the smallest difference
smallest_diff_feature = min(accuracy_differences, key=lambda x: x[1])

print(f"\nAccuracy Differences:")
for diff in accuracy_differences:
    print(diff)

print(f"\nFeature with the smallest impact: {smallest_diff_feature[0]} (Difference: {smallest_diff_feature[1]})")

# Filter the results to compare only 'age', 'balance', 'marital', 'previous'
specified_features = ('age', 'balance', 'marital', 'previous')
filtered_differences = [item for item in accuracy_differences if item[0].startswith(specified_features)]

# Find the specified feature with the smallest difference
smallest_diff_specified = min(filtered_differences, key=lambda x: x[1])

print(f"\nAmong the specified features, the one with the smallest impact is: {smallest_diff_specified[0]} (Difference: {smallest_diff_specified[1]})")




Accuracy Differences:
('age', 0.0003)
('balance', 0.0002)
('campaign', 0.0007)
('contact=cellular', 0.0007)
('contact=telephone', 0.0009)
('contact=unknown', 0.0007)
('day', 0.0006)
('duration', 0.0129)
('education=primary', 0.0009)
('education=secondary', 0.0003)
('education=tertiary', 0.0002)
('education=unknown', 0.001)
('housing=no', 0.0007)
('housing=yes', 0.0006)
('job=admin.', 0.0002)
('job=blue-collar', 0.0008)
('job=entrepreneur', 0.0006)
('job=housemaid', 0.0006)
('job=management', 0.0003)
('job=retired', 0.0004)
('job=self-employed', 0.0008)
('job=services', 0.0006)
('job=student', 0.0001)
('job=technician', 0.0011)
('job=unemployed', 0.0001)
('job=unknown', 0.0007)
('marital=divorced', 0.0006)
('marital=married', 0.0004)
('marital=single', 0.0009)
('month=apr', 0.0006)
('month=aug', 0.0006)
('month=dec', 0.0008)
('month=feb', 0.0011)
('month=jan', 0.0004)
('month=jul', 0.0)
('month=jun', 0.0002)
('month=mar', 0.0002)
('month=may', 0.0001)
('month=nov', 0.0006)
('month=oct'

In [102]:
# # Question 6
# Now let's train a regularized logistic regression.
# Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
# Train models using all the features as in Q4.
# Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
# Which of these C leads to the best accuracy on the validation set?

# Define the values of the parameter C to try
C_values = [0.01, 0.1, 1, 10, 100]

# Store the accuracies for each value of C
accuracy_results = []

for C in C_values:
    # Train the logistic regression model with the specified parameter C
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train_encoded, Y_train)

    # Calculate the accuracy on the validation dataset
    accuracy = model.score(X_val_encoded,Y_val)
    accuracy_results.append((C, round(accuracy, 3)))

    print(f"C = {C}, Validation Accuracy: {round(accuracy, 3)}")

# Find the C value that leads to the best accuracy on the validation set
best_C, best_accuracy = max(accuracy_results, key=lambda x: x[1])
print(f"\nThe best C value is: {best_C} with a validation accuracy of {best_accuracy}")




C = 0.01, Validation Accuracy: 0.898
C = 0.1, Validation Accuracy: 0.901
C = 1, Validation Accuracy: 0.902
C = 10, Validation Accuracy: 0.901
C = 100, Validation Accuracy: 0.901

The best C value is: 1 with a validation accuracy of 0.902
